In [ ]:
# -*- coding: utf-8 -*-

import time
import heapq
import numpy as np
from scipy import optimize
from scipy.optimize import fsolve
from concurrent.futures import ProcessPoolExecutor, as_completed
import matplotlib.pyplot as plt
import os
import pandas as pd
import geopandas as gpd 

inputLocation = "Taipei/"
CPU_COUNT = os.cpu_count() or 1
print(f"偵測到電腦核心數 = {CPU_COUNT}")

# -------------------------------
# 基本資料結構與初始化
# -------------------------------
class Zone:
    def __init__(self, _tmpIn):
        self.zoneId = _tmpIn[0]
        self.lat = 0
        self.lon = 0
        self.destList = []

class Node:
    '''
    此類別儲存網絡中節點的資訊
    '''
    def __init__(self, _tmpIn):
        self.Id = _tmpIn[0]
        self.lat = 0
        self.lon = 0
        self.outLinks = []
        self.inLinks = []
        self.label = float("inf")
        self.pred = ""
        self.inDegree = 0
        self.outDegree = 0
        self.order = 0  # 拓樸排序順序
        self.wi = 0.0   # Dial 演算法中節點權重
        self.xi = 0.0   # Dial 演算法中節點流量
        self.xj = 0.0   # 用於流量回溯時累計需求

class Link:
    '''
    此類別儲存網絡中連結的資訊
    flowCar 與 flowScooter 用來分別儲存汽車與機車的流量
    '''
    def __init__(self, _tmpIn):
        self.tailNode = _tmpIn[0]
        self.headNode = _tmpIn[1]
        self.capacity = float(_tmpIn[2])  # veh per hour
        self.length = float(_tmpIn[3])   # Length
        self.fft = float(_tmpIn[4])      # Free flow travel time (min)
        self.beta = float(_tmpIn[6])
        self.alpha = float(_tmpIn[5])
        self.speedLimit = float(_tmpIn[7])
        if len(_tmpIn) > 10:
            self.NMoto = int(_tmpIn[9])
            self.NCar = int(_tmpIn[10])
        else:
            self.NMoto = 0
            self.NCar = 0
            print(f'警告：連結 {_tmpIn[0]} -> {_tmpIn[1]} 沒有機車或汽車數量資訊，預設為 0')

        self.flowCar = 0.0
        self.flowScooter = 0.0
        self.cost = float(_tmpIn[4])
        self.logLike = 0.0
        self.reasonable = True
        self.wij = 0.0
        self.xij = 0.0

class Demand:
    '''
    讀取需求時，額外儲存 vehType (1=汽車, 2=機車)
    '''
    def __init__(self, _tmpIn):
        self.fromZone = _tmpIn[0]
        self.toNode = _tmpIn[1]
        self.demand = float(_tmpIn[2])
        self.vehType = _tmpIn[3]  # '1' or '2'

def readDemand():
    '''
    讀取需求檔案，需求檔結構假設:
    fromZone  toZone  demand  vehType
    其中 vehType = '1' (汽車) or '2' (機車)
    '''
    inFile = open(inputLocation + f"{year}{rush_hour}_demand.dat", encoding="utf-8")
    _ = inFile.readline().strip().split("\t")  # 讀掉標頭
    for x in inFile:
        tmpIn = x.strip().split("\t")
        demandObj = Demand(tmpIn)
        tripSet[(tmpIn[0], tmpIn[1], tmpIn[3])] = demandObj

        if tmpIn[0] not in zoneSet:
            zoneSet[tmpIn[0]] = Zone([tmpIn[0]])
        if tmpIn[1] not in zoneSet:
            zoneSet[tmpIn[1]] = Zone([tmpIn[1]])
        if tmpIn[1] not in zoneSet[tmpIn[0]].destList:
            zoneSet[tmpIn[0]].destList.append(tmpIn[1])

        originZones.add((tmpIn[0], tmpIn[3]))

    inFile.close()
    print(len(tripSet), "OD pairs (含車種)")
    print(len(zoneSet), "zones")

def readNetwork():
    inFile = open(inputLocation + f"Net_modified_{scenario}_{rush_hour}調撥.dat", encoding="utf-8")
    tmpIn = inFile.readline().strip().split("\t")  # 讀掉標頭
    for x in inFile:
        tmpIn = x.strip().split("\t")
        linkSet[(tmpIn[0], tmpIn[1])] = Link(tmpIn)
        if tmpIn[0] not in nodeSet:
            nodeSet[tmpIn[0]] = Node([tmpIn[0]])
        if tmpIn[1] not in nodeSet:
            nodeSet[tmpIn[1]] = Node([tmpIn[1]])
        if tmpIn[1] not in nodeSet[tmpIn[0]].outLinks:
            nodeSet[tmpIn[0]].outLinks.append(tmpIn[1])
        if tmpIn[0] not in nodeSet[tmpIn[1]].inLinks:
            nodeSet[tmpIn[1]].inLinks.append(tmpIn[0])
    inFile.close()
    print(len(nodeSet), "nodes")
    print(len(linkSet), "links")

def readTurnRestrictions(filename="no_left_turns.dat"):
    '''
    no_left_turns.dat 假設五欄: A B C set pen
    pen = -1 -> 禁止通行
    pen = 2  -> 機車兩段式左轉 +1分鐘
    pen = 3  -> 機車兩段式左轉(大型路口) +2分鐘
    '''
    global turn_restrict
    try:
        with open(inputLocation + filename) as inFile:
            for line in inFile:
                parts = line.strip().split()
                if len(parts) == 5:
                    A, B, C, vehSet, pen = parts
                    turn_restrict[(A, B, C, vehSet)] = pen
        print(f"成功讀取 {len(turn_restrict)} 筆轉向限制/懲罰資料")
    except FileNotFoundError:
        print(f"警告：未找到禁止轉向檔案 {filename}，將不施加任何限制。")

def DijkstraHeap(origin, vehType):
    '''
    多車種 Dijkstra，若遇到禁行機車 (NMoto=1) 會跳過
    '''
    for n in nodeSet:
        nodeSet[n].label = float("inf")
        nodeSet[n].pred = ""
    nodeSet[origin].label = 0.0
    nodeSet[origin].pred = "NA"
    SE = [(0, origin)]
    while SE:
        currentNode = heapq.heappop(SE)[1]
        currentLabel = nodeSet[currentNode].label
        prevNode = nodeSet[currentNode].pred
        for toNode in nodeSet[currentNode].outLinks:
            addPenalty = 0.0
            if vehType == '2' and linkSet[(currentNode, toNode)].NMoto == 1:
                continue
            if vehType == '1' and linkSet[(currentNode, toNode)].NCar == 1:
                continue
            if prevNode and prevNode != "NA":
                trKey = (prevNode, currentNode, toNode, vehType)
                if trKey in turn_restrict:
                    penVal = turn_restrict[trKey]
                    if penVal == '-1':
                        continue
                    else:
                        if vehType == '2':
                            if penVal == '2':
                                addPenalty = 2.0
                            elif penVal == '3':
                                addPenalty = 3.0

            link = (currentNode, toNode)
            newLabel = currentLabel + linkSet[link].cost + addPenalty
            existingLabel = nodeSet[toNode].label
            if newLabel < existingLabel:
                nodeSet[toNode].label = newLabel
                nodeSet[toNode].pred = currentNode
                heapq.heappush(SE, (newLabel, toNode))

def tracePreds(dest):
    '''
    回溯最短路徑，若偵測到 pred 為空或 (prevNode, dest) 不在 linkSet 中就提早停止
    '''
    spLinks = []
    visited = set()
    while True:
        prevNode = nodeSet[dest].pred
        if prevNode == "NA":
            break
        if dest in visited:
            print(f"循環偵測：節點 {dest} 無法正確回溯")
            break
        visited.add(dest)
        if (not prevNode) or ((prevNode, dest) not in linkSet):
            print(f"警告：節點 {dest} 的 pred={prevNode} 不存在對應 link，停止回溯。")
            break
        spLinks.append((prevNode, dest))
        dest = prevNode
    return spLinks

def updateTravelTime():
    '''
    使用 BPR formula 更新 travel time
    '''
    for l in linkSet:
        totalFlow = linkSet[l].flowCar + linkSet[l].flowScooter
        ratio = totalFlow / linkSet[l].capacity if linkSet[l].capacity != 0 else 0
        linkSet[l].cost = linkSet[l].fft * (1 + linkSet[l].alpha * (ratio ** linkSet[l].beta))

def findAlpha(x_bar_car, x_bar_scooter):
    '''
    使用 Frank-Wolfe 的步長解
    '''
    def df(alpha):
        sum_derivative = 0
        for l in linkSet:
            tmpFlowCar = linkSet[l].flowCar + alpha * (x_bar_car[l] - linkSet[l].flowCar)
            tmpFlowScooter = linkSet[l].flowScooter + alpha * (x_bar_scooter[l] - linkSet[l].flowScooter)
            totalFlow = tmpFlowCar + tmpFlowScooter
            if linkSet[l].capacity != 0:
                tmpCost = linkSet[l].fft * (1 + linkSet[l].alpha * ((totalFlow / linkSet[l].capacity) ** linkSet[l].beta))
            else:
                tmpCost = 0.0
            sum_derivative += ((x_bar_car[l] - linkSet[l].flowCar) + (x_bar_scooter[l] - linkSet[l].flowScooter)) * tmpCost
        return sum_derivative

    sol = optimize.root(df, np.array([0.1]))
    sol2 = fsolve(df, np.array([0.1]))
    if not sol.success or not (0 < sol2[0] <= 1):
        print("fsolve 無法找到有效步長，改用 fallback 步長 = 0.2")
        return 0.2
    return max(0.1, min(1, sol2[0]))

def loadAON():
    '''
    多車種 All-or-Nothing: 針對每組 (originZone, vehType) 做最短路徑
    回傳:
        totalSPTT (Car + Scooter)
        x_bar_car
        x_bar_scooter
    '''
    x_bar_car = {l: 0.0 for l in linkSet}
    x_bar_scooter = {l: 0.0 for l in linkSet}
    totalSPTT = 0.0

    for (origin, vt) in originZones:
        DijkstraHeap(origin, vt)
        for s in zoneSet[origin].destList:
            if (not np.isfinite(nodeSet[s].label)) or nodeSet[s].label < 0:
                continue
            if (origin, s, vt) in tripSet:
                dem = tripSet[(origin, s, vt)].demand
            else:
                dem = 0.0
            totalSPTT += nodeSet[s].label * dem
            if origin != s:
                for spLink in tracePreds(s):
                    if vt == '1':
                        x_bar_car[spLink] += dem
                    else:
                        x_bar_scooter[spLink] += dem

    return totalSPTT, x_bar_car, x_bar_scooter

def writeUEresults():
    outFile = open(f"{year}_{scenario}_{rush_hour}_UE_results.dat", "w", encoding="utf-8")
    tmpOut = "tailNode\theadNode\tcapacity\tlength\tfft\tUE_travelTime\tUE_flow"
    outFile.write(tmpOut + "\n")
    for i in linkSet:
        tmpOut = f"{linkSet[i].tailNode}\t{linkSet[i].headNode}\t{linkSet[i].capacity}\t{linkSet[i].length}\t{linkSet[i].fft}\t{linkSet[i].cost}\t{linkSet[i].flowCar + linkSet[i].flowScooter}"
        outFile.write(tmpOut + "\n")
    outFile.close()

def assignment(loading, algorithm, accuracy, maxIter, allow, recal, parallel_loading=False):
    '''
    執行多車種交通指派時，若判定到 TSTT 或 SPTT 為 0 或非數值，就跳過避免 gap 計算出 NaN。
    '''
    mingap = False
    it = 1
    count = 0
    current_min_gap = float("inf")
    gap = float("inf")
    x_bar_car = {l: 0.0 for l in linkSet}
    x_bar_scooter = {l: 0.0 for l in linkSet}
    startP = time.time()
    gap_lst = []

    # 初始化都為 0
    for l in linkSet:
        linkSet[l].flowCar = 0.0
        linkSet[l].flowScooter = 0.0

    print("==== 開始多車種交通指派 ====")
    while gap > accuracy:
        if algorithm == "MSA" or it < 2:
            alpha = 1.0 / it
        elif algorithm == "FW":
            alpha = findAlpha(x_bar_car, x_bar_scooter)
        else:
            print("Terminating the program.....不明演算法:", algorithm)
            return

        prevCarFlow = np.array([linkSet[l].flowCar for l in linkSet])
        prevScooterFlow = np.array([linkSet[l].flowScooter for l in linkSet])

        for l in linkSet:
            linkSet[l].flowCar     = alpha * x_bar_car[l]     + (1 - alpha) * linkSet[l].flowCar
            linkSet[l].flowScooter = alpha * x_bar_scooter[l] + (1 - alpha) * linkSet[l].flowScooter

        updateTravelTime()

        if loading == "deterministic":
            SPTT, x_bar_car, x_bar_scooter = loadAON()
            TSTT = 0.0
            for a in linkSet:
                flow_sum = linkSet[a].flowCar + linkSet[a].flowScooter
                if not np.isfinite(linkSet[a].cost) or linkSet[a].cost < 0 or flow_sum < 0:
                    continue
                TSTT += linkSet[a].cost * flow_sum

            if (not np.isfinite(SPTT)) or (SPTT == 0):
                print("警告：SPTT 為 0 或非數值，將 gap 設為 0 並中止迭代。")
                gap = 0
            else:
                gap_val = (TSTT / SPTT) - 1
                if not np.isfinite(gap_val):
                    gap = float("inf")
                else:
                    gap = round(abs(gap_val), 5)

            if it == 1:
                gap = gap + float("inf")

        elif loading == "stochastic":
            print("尚未實作多車種的 DialLoad，此處可自行擴充")
            return
        else:
            print("Terminating the program..... 不明 loading:", loading)
            return

        gap_lst.append(gap)

        if gap < current_min_gap and np.isfinite(gap):
            current_min_gap = gap
            count = 0
            writeUEresults()
            print("Iteration", it, "improved gap:", gap, "count reset to 0")
        else:
            if count > allow:
                print("The assignment did not converge: no improvement in", allow, "iterations, current gap:", gap)
                mingap = True
                break
            count += 1
            print("Iteration", it, "no improvement in gap:", gap, "count:", count)

        if it >= maxIter:
            print("The assignment did not converge: max iterations reached, current gap:", gap)
            break

        print(f"    Iteration {it} completed :)")
        it += 1

    print("Assignment took", time.time() - startP, "seconds")
    print("Assignment stopped in", it, "iterations, current gap:", gap)
    min_gap = min(gap_lst) if gap_lst else float("inf")
    min_gap_it = gap_lst.index(min_gap) + 1 if gap_lst else 0
    print("Minimum gap found at iteration", min_gap_it, "with gap", min_gap)

    if not recal:
        plt.plot(range(1, len(gap_lst) + 1), gap_lst, marker='o')
        plt.xlabel("Iteration")
        plt.ylabel("Gap")
        plt.title("Multi-Class Gap vs. Iteration")
        plt.grid()
        plt.savefig("gap_vs_iteration.png")
        plt.show()

def checkNetworkHealth():
    print("==== Network Health Check ====")
    isolated = [n for n in nodeSet if len(nodeSet[n].inLinks) == 0 and len(nodeSet[n].outLinks) == 0]
    print(f"完全孤立節點: {len(isolated)}")
    unreachable = [node for node in nodeSet if nodeSet[node].pred == ""]
    print(f"目前未標定 predecessor 的節點數: {len(unreachable)}")
    allFlowZero = all((linkSet[l].flowCar + linkSet[l].flowScooter) == 0.0 for l in linkSet)
    print("所有流量皆為 0？", allFlowZero)

def savetoShapefile():
    cap_dict = {}
    with open('TRTS4S_Capacity.csv', 'r') as f:
        lines = f.readlines()
        for line in lines[1:]:
            parts = line.strip().split(',')
            class_num = int(parts[0])
            for i in range(1, len(parts)):
                lanes = i
                capacity = int(parts[i])
                cap_dict[(class_num, lanes)] = capacity
    ue_results = pd.read_csv(f"{year}_{scenario}_{rush_hour}_UE_results.dat", sep="\t")
    gdf = gpd.read_file(f"Network/Net_modified_{scenario}_{rush_hour}調撥.shp")
    ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)
    gdf = gdf.merge(ue_results[["A", "B", "capacity", "UE_travelTime", "UE_flow"]], on=["A", "B"], how="left")
    gdf['capacity'] = gdf.apply(lambda row: get_capacity(row, cap_dict), axis=1)
    gdf["V/C"] = gdf["UE_flow"] / gdf["capacity"]
    gdf["assigned_speed"] = gdf["LENGTH"] / gdf["UE_travelTime"] * 60 
    gdf.to_file(f"{year}_{scenario}_{rush_hour}_UE_results.shp")

def get_capacity(row, cap_dict):
    class_num = row['CLASS_110']
    lanes = row['LANES_110']
    if (class_num, lanes) in cap_dict:
        return cap_dict[(class_num, lanes)]
    else:
        print(f"Warning: No capacity found for class {class_num} and lanes {lanes}. Using default value 0.")
        return 0

year = 104
for scenario in ['現況', '全線拆除', '部分拆除']:
    for rush_hour in ['晨峰', '昏峰']:
        tripSet = {}
        zoneSet = {}
        linkSet = {}
        nodeSet = {}
        turn_restrict = {}
        originZones = set()

        readStart = time.time()
        readDemand()
        readNetwork()
        readTurnRestrictions()
        print("Reading the network data took", round(time.time() - readStart, 2), "secs")

        assignment(
            loading="deterministic",
            algorithm="FW",
            accuracy=0.001,
            maxIter=50,
            allow=10,
            recal=False,
            parallel_loading=True
        )

        checkNetworkHealth()
        savetoShapefile()